<h2>Problem</h2> 
Every website requires a registration for basic tasks
<h2>Solution</h2>
Create a pseudorandom password generator

In [6]:
import string, random
def generate_password(length):
    password = ""
    for i in range(length):
        choices = string.ascii_letters + string.digits + string.punctuation
        password += random.choice(choices)
    return password

## Mongodb Local Logging

In [27]:
from pymongo import MongoClient
def log_password(username, password, url):
    client = MongoClient("localhost", 27017)
    random_db = client.random_passwords
    random_col = random_db["rng_passwords"]
    random_col.insert_one({"Username": username, 
                           "Password": password,
                           "Url":      url})
    client.close()

## Testing of logging

In [8]:
password = generate_password(10)
print("Randomly generated password is:", password)
log_password("johndoe@gmail.com", password, "https://my.stuff.co.nz")

Randomly generated password is: F.{tk<">R>


## Create function to retrieve a record

In [28]:
def retrieve(username, url=None):
    client = MongoClient("localhost", 27017)
    random_db = client.random_passwords
    random_col = random_db["rng_passwords"]
    if url is not None:
        return random_col.find_one({"Username": username, "Url": url})
    client.close()
    return random_col.find({"Username": username})

## Test encapsulation

In [10]:
def test_password_entry(username, password, url):
    user_record = retrieve(username, url)
    if user_record is None:
        return False
    return user_record["Password"] == password

test_password_entry("johndoe@gmail.com", password, "https://my.stuff.co.nz")

False

## Send generated password to the clipboard

In [11]:
import pyperclip
pyperclip.copy(generate_password(10))
print(pyperclip.paste())

;K$3,b-_&P


## Bundle all functionality together

In [12]:
import pyperclip
def password_package(username, url, length=10):
    password = generate_password(length)
    log_password(username, password, url)
    if not test_password_entry(username, password, url):
        raise Exception("Logging has failed, check db")
    pyperclip.copy(password)
    print("Password has been generated, logged, and sent to clipboard.")

## Testing of package

In [20]:
password_package("big_boy@gmail.com", "reddit.com")

Password has been generated, logged, and sent to clipboard.


In [23]:
result = retrieve(username="big_boy@gmail.com", url="reddit.com")
password = result['Password']
url = result['Url']
username = result['Username']
test_password_entry(username, password, url)

True

<h2>Problem</h2>
Websites have varying restrictions on passwords, for example some sites require at least one capital letter, number and punctuation character.
<h2>Solution</h2>
Make a function for randomly altering the generated password to conform with restrictions provided.

In [35]:
def insert(string, char, pos):
    return string[:pos] + char + string[pos:]

def restrict(password, capitals=0, numbers=0, punctuation=0):
    if capitals == 0 and numbers == 0 and punctuation == 0:
        raise Error("At least one restriction must be provided")
    for i in range(capitals):
        cap = random.choice(string.ascii_uppercase)
        password = insert(password, cap, random.randint(1, len(password)-1))
    for j in range(numbers):
        num = random.choice(string.digits)
        password = insert(password, num, random.randint(1, len(password)-1))
    for k in range(punctuation):
        punc = random.choice(string.punctuation)
        password = insert(password, punc, random.randint(1, len(password)-1))
    return password

## Testing for restriction

In [38]:
print("Generated password\t\tRestricted Password")
for i in range(5):
    password = generate_password(5)
    print(password + "\t\t\t" + restrict(password, capitals=4))

Generated password		Restricted Password
js9yn			jKMWCs9yn
7Lq*N			7OLqN*MSN
8i4]T			8PiK4AN]T
LG,>m			LLPAG,O>m
sAKf\			sAKKfYNL\
